In [2]:
#had problems so put it here just for now
from pysnptools.distreader._bgen2 import bgen_reader2, Bgen2

In [3]:
filename = r'D:\OneDrive\Shares\bgenreaderpy\1x1000000.bgen'

In [2]:
from bgen_reader import read_bgen

In [3]:
%%time #!!!cmk memory use goes up to 4GB!
bgen = read_bgen(filename) #cmk: didn't say verbose, but was anyway. #cmk much of docs is blank

We will create the metafile `D:\OneDrive\Shares\bgenreaderpy\1x1000000.metadata`. This file will speed up further
reads and only need to be created once. So, please, bear with me.


Mapping genotypes: 100%|██████████████████████████████████████████████████| 1000000/1000000 [00:40<00:00, 24824.73it/s]

Wall time: 43.6 s


In [4]:
%%time
bgen = read_bgen(filename) #cmk reading a second time with metadata there takes another 48 seconds

Mapping genotypes: 100%|██████████████████████████████████████████████████| 1000000/1000000 [00:43<00:00, 22971.93it/s]


Wall time: 44.4 s


In [4]:
%%time
#We can use
#with bgen_reader2(filename) as bgen2:
#or
bgen2 = Bgen2(filename)
print(bgen2.id[:5]) #other property arrays include risd,chrom,position,nallels, and allele_ids

['sid_0' 'sid_1' 'sid_2' 'sid_3' 'sid_4']
Wall time: 8.46 s


In [5]:
%%time
#second time we read, it is faster
bgen2 = Bgen2(filename)
print(bgen2.id[:5])

['sid_0' 'sid_1' 'sid_2' 'sid_3' 'sid_4']
Wall time: 354 ms


In [5]:
%%time
variant_count = len(bgen["genotype"])
geno = bgen["genotype"][variant_count-1].compute()
geno["probs"]

Wall time: 36 ms


array([[0.64705882, 0.07843137, 0.2745098 ]])

In [7]:
%%time
#read one variant
print(bgen2.read(-1))

[[[0.64705882 0.07843137 0.2745098 ]]]
Wall time: 999 µs


In [6]:
%%time
#!!!cmk memory use about 2GB
import numpy as np
probs = [geno.compute()['probs'] for geno in bgen["genotype"][:5]]
print(np.array(probs)) #!!!cmk extra dimension of array (could likely be fixed with better numpy)

chrom5_indexes = np.arange(variant_count)[np.array(bgen["variants"]['chrom']=='5')]
print(len(chrom5_indexes)) #len is 63336
probs = [bgen["genotype"][index].compute()['probs'] for index in chrom5_indexes] #Just 10 of them
print(np.array(probs))

UsageError: Can't use statement directly after '%%time'!


In [8]:
%%time
print(bgen2.read(slice(5))) #read the first 5
print(bgen2.read(bgen2.chrom=='5',max_ncombs=4)) #read chrom 5, allow more combos

[[[0.46666667 0.38823529 0.14509804]
  [0.48235294 0.10588235 0.41176471]
  [0.7372549  0.11372549 0.14901961]
  [0.28627451 0.10196078 0.61176471]
  [0.05490196 0.38431373 0.56078431]]]
[[[       nan        nan        nan        nan]
  [0.65098039 0.01960784 0.32941176        nan]
  [0.37254902 0.36078431 0.26666667        nan]
  ...
  [0.16862745 0.67843137 0.15294118        nan]
  [       nan        nan        nan        nan]
  [0.40784314 0.18823529 0.40392157        nan]]]
Wall time: 794 ms


In [11]:
filename = r'g:\deldir\1000x500000.bgen' #Bigger file

In [9]:
%%time
#!!!cmk memory use goes up to 2.1GB
bgen = read_bgen(filename)

We will create the metafile `g:\deldir\1000x500000.metadata`. This file will speed up further
reads and only need to be created once. So, please, bear with me.


Mapping genotypes: 100%|████████████████████████████████████████████████████| 500000/500000 [00:22<00:00, 21812.93it/s]


Wall time: 3min 21s


In [3]:
%%time
bgen = read_bgen(filename) #2nd time

Mapping genotypes: 100%|████████████████████████████████████████████████████| 500000/500000 [00:20<00:00, 24585.47it/s]

Wall time: 20.4 s


In [12]:
%%time
bgen2 = Bgen2(filename)
print(bgen2.id[:5]) 

['sid_0' 'sid_1' 'sid_2' 'sid_3' 'sid_4']
Wall time: 25.7 s


In [13]:
%%time
#faster after the first time

bgen2 = Bgen2(filename)
print(bgen2.id[:5]) 

['sid_0' 'sid_1' 'sid_2' 'sid_3' 'sid_4']
Wall time: 153 ms


In [5]:
%%time
import numpy as np
#read 1000 samples x 1000 variants and average them
probs = [geno.compute()['probs'] for geno in bgen["genotype"][-1000:]]
print(np.nanmean(np.array(probs)))

0.3333333333333333
Wall time: 4.03 s


In [14]:
%%time
import numpy as np

#read 1000 samples x 1000 variants and average them
print(np.nanmean(bgen2.read(np.s_[-1000:])))

0.333333333333333
Wall time: 235 ms


In [6]:
%%time

import pandas as pd #Uses 1.2 GB of memory

#Find the mean distribution for every 50th variant and create a Pandas dataframe
mean_per_variant = np.array([np.nanmean(geno.compute()['probs'],axis=0) for geno in bgen["genotype"][::50]])
indexes = np.arange(len(bgen["genotype"]))[::50] #This is fast
idx = bgen["variants"].loc[indexes,'id'].compute()
df = pd.DataFrame({'id':idx,'p0':mean_per_variant[:,0],'p1':mean_per_variant[:,1],'p2':mean_per_variant[:,2],})
df

Wall time: 3min 15s


,id,p0,p1,p2
0,sid_0,0.327895,0.336770,0.335335
50,sid_50,0.321633,0.332449,0.345918
100,sid_100,0.334635,0.331595,0.333770
150,sid_150,0.327974,0.337406,0.334621
200,sid_200,0.337305,0.322352,0.340342
...,...,...,...,...
499750,sid_499750,0.341511,0.330225,0.328263
499800,sid_499800,0.331892,0.328181,0.339927
499850,sid_499850,0.332797,0.336960,0.330243
499900,sid_499900,0.337645,0.332883,0.329471


In [16]:
%%time
import pandas as pd

#Find the mean distribution for every 50th variant and create a Pandas dataframe

mean_per_variant = np.nanmean(bgen2.read(np.s_[::50]),axis=0)
df = pd.DataFrame({'id':bgen2.id[::50],'p0':mean_per_variant[:,0],'p1':mean_per_variant[:,1],'p2':mean_per_variant[:,2],})
df

Wall time: 2.4 s


,id,p0,p1,p2
0,sid_0,0.327895,0.336770,0.335335
1,sid_50,0.321633,0.332449,0.345918
2,sid_100,0.334635,0.331595,0.333770
3,sid_150,0.327974,0.337406,0.334621
4,sid_200,0.337305,0.322352,0.340342
...,...,...,...,...
9995,sid_499750,0.341511,0.330225,0.328263
9996,sid_499800,0.331892,0.328181,0.339927
9997,sid_499850,0.332797,0.336960,0.330243
9998,sid_499900,0.337645,0.332883,0.329471
